In [ ]:
from humpback import *

from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [ ]:
pd.options.mode.copy_on_write = True

## Fetch Data

In [ ]:
symbol = 'BTCUSDT'

train_size, val_size, test_size = 0.5, 0.2, 0.3

input_sequence_length = 20

In [ ]:
data_raw_df = pd.read_csv(f'Binance Data/{symbol}.csv', index_col='index')

In [ ]:
data_return = getReturn(data_raw_df, 'Close')
data_feature = featureGeneration(data_return)
X_data_df , y_data_df  = getXy(data_feature, 'Return')

In [ ]:
X_train_array, X_val_array, X_test_array, X_scaler, X_pca = preprocess(X_data_df, train_size, val_size, test_size, apply_PCA=True, PCA_components=6)
y_train_array, y_val_array, y_test_array, y_scaler = preprocess(y_data_df, train_size, val_size, test_size, apply_PCA=False)

In [ ]:
train_dataset = keras.utils.timeseries_dataset_from_array(
    X_train_array,
    y_train_array,
    sequence_length=input_sequence_length,
    shuffle=False)

In [ ]:
val_dataset = keras.utils.timeseries_dataset_from_array(
    X_val_array,
    y_val_array,
    sequence_length=input_sequence_length,
    shuffle=False) 

In [ ]:
test_dataset = keras.utils.timeseries_dataset_from_array(
    X_test_array,
    None,
    sequence_length=input_sequence_length,
    shuffle=False) 

## Model

In [ ]:
class LSTM_stack(layers.Layer):
    """stacks of LSTM with dropout, follwed by dense layer"""

    def __init__(
        self,
        dropout_rate: float = 0.2
        ):
        super().__init__()

        self.lstm_1 = layers.LSTM(
            units=50,
            return_sequences=True,
            )

        self.lstm_2 = layers.LSTM(
            units=50,
            return_sequences=True,
            )

        self.lstm_3 = layers.LSTM(
            units=50,
            return_sequences=False,
            )

        self.dense_1 = layers.Dense(
            units=1
            )

        self.dropout_1 = layers.Dropout(dropout_rate)

    def call(self, inputs):
        x = self.lstm_1(inputs)
        x = self.dropout_1(x)
        x = self.lstm_2(inputs)
        x = self.dropout_1(x)
        x = self.lstm_3(inputs)
        x = self.dropout_1(x)
        return self.dense_1(x)

In [ ]:
inputs  = keras.Input((
    input_sequence_length,
    X_train_array.shape[1]))
outputs = LSTM_stack()(inputs)
model   = keras.Model(inputs, outputs)

In [ ]:
optimizer = keras.optimizers.Adam()
loss      = keras.losses.MeanSquaredError()

model.compile(optimizer=optimizer,loss=loss)

In [ ]:
model.fit(
    train_dataset,
    epochs=20,
    validation_data=val_dataset)

In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
plt.plot(y_scaler.inverse_transform(y_test_array[input_sequence_length-1:]), color='red', label='Actual')
plt.plot(y_scaler.inverse_transform(predictions), color='blue', label='Predicted')
plt.title('LSTM BTC 1h')
plt.legend() 

# Ad hoc

In [20]:
list(test_dataset)[-1][0].numpy().shape

2024-08-13 01:53:11.078229: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


(20, 6)